# 2. Develop your segmentation model

In [ ]:
#Import libraries
import os
from model_class import ModelSegmentation
from aux_functions import slicing
import cv2
#Inputs
working_directory="C:/Users/Pheno/Documents/database_almondcv3/"
pictures_directory=os.path.join(working_directory, "hyp_images_for_train")
pre_model=os.path.join(working_directory, "models/yolo11s-seg.pt")
model_path=os.path.join(working_directory, "")
output_directory=os.path.join(working_directory,"output_directory")

### Slicing

In [ ]:
#Slice_pictures for training
slicing(input_folder=pictures_directory,output_directory=working_directory,name_slicing="Slices_hyp", number_pictures=20, slice_height=320, slice_width=320)

### Training

In [3]:
#Label with CVAT

zip_file_shell=os.path.join(working_directory,"hyp_almonds.zip")

In [4]:
#Model segmentation training

model=ModelSegmentation(working_directory=working_directory)
model.train_segmentation_model(input_zip=zip_file_shell, epochs=100,imgsz=320, name_segmentation="hyp_almonds",
                                      pre_model=pre_model, batch=16)

Detected GPU: NVIDIA GeForce RTX 3060
Total GPU Memory: 12.00 GB
New https://pypi.org/project/ultralytics/8.3.80 available  Update with 'pip install -U ultralytics'
engine\trainer: task=segment, mode=train, model=C:/Users/Pheno/Documents/database_almondcv3/models/yolo11s-seg.pt, data=C:/Users/Pheno/Documents/database_almondcv3/hyp_almonds\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=C:/Users/Pheno/Documents/database_almondcv3/results_models_segmentation_hyp_almonds, name=results_training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, 

100%|██████████| 5.35M/5.35M [00:01<00:00, 5.54MB/s]


AMP: checks passed 


train: Scanning C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\labels\Train... 464 images, 126 backgrounds, 0 corrupt: 100%|██████████| 590/590 [00:00<00:00, 800.83it/s]

train: WARNING  C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\images\Train\SL_HYP_leaf_jijona_cosecha2024_3_2025-02-13-10-51-46_0_512_2048_832_2368.png: 1 duplicate labels removed
train: WARNING  C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\images\Train\SL_HYP_leaf_jijona_cosecha2024_3_2025-02-13-10-51-46_0_512_3328_832_3648.png: 1 duplicate labels removed
train: New cache created: C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\labels\Train.cache


...

                   all        200        204      0.985      0.992      0.995      0.984      0.985      0.992      0.995      0.956



100 epochs completed in 0.165 hours.
Optimizer stripped from C:\Users\Pheno\Documents\database_almondcv3\results_models_segmentation_hyp_almonds\results_training\weights\last.pt, 20.5MB
Optimizer stripped from C:\Users\Pheno\Documents\database_almondcv3\results_models_segmentation_hyp_almonds\results_training\weights\best.pt, 20.5MB

Validating C:\Users\Pheno\Documents\database_almondcv3\results_models_segmentation_hyp_almonds\results_training\weights\best.pt...
Ultralytics 8.3.18  Python-3.12.2 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLO11s-seg summary (fused): 265 layers, 10,067,203 parameters, 0 gradients, 35.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.18it/s]


                   all        200        204      0.989      0.995      0.995      0.981      0.989      0.995      0.995      0.968
Speed: 0.3ms preprocess, 1.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to C:\Users\Pheno\Documents\database_almondcv3\results_models_segmentation_hyp_almonds\results_training

image 1/200 C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\images\Test\SL_HYP_SEED_shell_16_08_1_2024-08-16-09-18-42_0_0_0_320_320.png: 320x320 1 almonds, 14.1ms
image 2/200 C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\images\Test\SL_HYP_SEED_shell_16_08_1_2024-08-16-09-18-42_0_0_1024_320_1344.png: 320x320 2 almondss, 14.0ms
image 3/200 C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\images\Test\SL_HYP_SEED_shell_16_08_1_2024-08-16-09-18-42_0_0_1280_320_1600.png: 320x320 1 almonds, 15.1ms
image 4/200 C:\Users\Pheno\Documents\database_almondcv3\hyp_almonds\images\Test\SL_HYP_SEED_shell_16_08_1_2024-08-16-09-18-42_0_0_1536_320_1856.

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'almonds'}
 obb: None
 orig_img: array([[[38, 38, 38],
         [38, 38, 38],
         [37, 37, 37],
         ...,
         [56, 56, 56],
         [53, 53, 53],
         [55, 55, 55]],
 
        [[39, 39, 39],
         [38, 38, 38],
         [38, 38, 38],
         ...,
         [55, 55, 55],
         [52, 52, 52],
         [53, 53, 53]],
 
        [[39, 39, 39],
         [39, 39, 39],
         [38, 38, 38],
         ...,
         [55, 55, 55],
         [51, 51, 51],
         [53, 53, 53]],
 
        ...,
 
        [[37, 37, 37],
         [37, 37, 37],
         [37, 37, 37],
         ...,
         [53, 53, 53],
         [52, 52, 52],
         [52, 52, 52]],
 
        [[38, 38, 38],
         [38, 38, 38],
         [37, 37, 37],
         ...,
         [53, 53, 53],
         [51, 51, 51],
         [51, 51,

### Deploy and reconstruct a picture

#### Slice_predict_reconstruct approach

In [ ]:
# Join patches approach

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=320, model_path=model_path,
                                          slice_height=640, slice_width=640,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)

In [ ]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    cv2.imwrite(f"{output_directory}/{os.path.basename(mask[1])}", mask[0])


#### SAHI

In [ ]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=640, slice_width=640,
                                                  confidence_treshold=0.8,
                                                  imgsz=640)

In [ ]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    mask[0].export_visuals(export_dir=output_directory, hide_labels=True, rect_th=1, file_name=f"{os.path.basename(mask[1])}")